# Set an Environment

In [1]:
%cd drive/MyDrive/Projects/AFTER

/content/drive/MyDrive/Projects/AFTER


In [2]:
import pandas as pd
import numpy as np
import jax.numpy as jnp

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# Current Quater

## Load Data

In [4]:
pred_now = pd.read_excel('pred_now.xlsx', index_col='TIME')

In [5]:
pred_now.index = pd.PeriodIndex(pred_now.index, freq='M')

In [6]:
pred_now = pred_now.asfreq('Q-DEC')

In [7]:
errors_now = pd.read_excel('error_now.xlsx', index_col='TIME')

In [8]:
errors_now.index = pd.PeriodIndex(errors_now.index, freq='M')

In [9]:
errors_now = errors_now.asfreq('Q-DEC')

In [28]:
true = pd.read_excel('GDP_true.xlsx', index_col='TIME')

In [29]:
true.index = pd.PeriodIndex(true.index, freq='M')

In [30]:
true = true.asfreq('Q-DEC')

In [33]:
true = true.reindex(errors_now.index)

In [13]:
true = true['GDP(SA)']

## Make Weights

In [14]:
weights = pd.DataFrame(index=errors_now.index, columns=errors_now.columns)

In [15]:
weights.loc[:'2016Q4'] = 1 / len(weights.columns)

In [16]:
weights

,DFM,BEM1,BEM2,BEM3,LBVAR,TFM,T-M,VAR,SVR-KNN,CRNN,RF-DVS,S-XGB,LQR,OptimARIMA
TIME,,,,,,,,,,,,,,
2016Q1,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429
2016Q2,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429
2016Q3,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429
2016Q4,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429
2017Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
for i in range(4, len(errors_now)):
  weights.iloc[i] = (
      (weights.iloc[i-1] / errors_now.iloc[:i].std()) * np.exp(-errors_now.iloc[i-1]**2 / (2 * errors_now.iloc[:i].var()))
      / np.sum((weights.iloc[i-1] / errors_now.iloc[:i].std()) * np.exp(-errors_now.iloc[i-1]**2 / (2 * errors_now.iloc[:i].var())))
  )


In [18]:
weights

,DFM,BEM1,BEM2,BEM3,LBVAR,TFM,T-M,VAR,SVR-KNN,CRNN,RF-DVS,S-XGB,LQR,OptimARIMA
TIME,,,,,,,,,,,,,,
2016Q1,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429
2016Q2,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429
2016Q3,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429
2016Q4,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429
2017Q1,0.001253,0.191632,0.22476,0.163671,0.089,0.011672,0.044676,0.017636,0.007862,0.014848,0.133874,0.000142,0.092113,0.006862
2017Q2,0.001335,0.188465,0.277516,0.252277,0.030557,0.010594,0.048585,0.014109,0.00557,0.008361,0.087733,0.000129,0.073294,0.001475
2017Q3,0.000935,0.011956,0.041197,0.204618,0.058362,0.025871,0.12169,0.035673,0.008988,0.022234,0.305019,0.000036,0.162762,0.000658
2017Q4,0.000188,0.001663,0.006362,0.036036,0.37777,0.01514,0.063394,0.14963,0.03198,0.009158,0.15092,0.00001,0.154087,0.003662
2018Q1,0.000057,0.001663,0.015754,0.035258,0.0683,0.005338,0.01584,0.069945,0.046318,0.003178,0.05168,0.0,0.68263,0.004038


In [19]:
weights.sum(axis=1)

TIME
2016Q1    1.0
2016Q2    1.0
2016Q3    1.0
2016Q4    1.0
2017Q1    1.0
2017Q2    1.0
2017Q3    1.0
2017Q4    1.0
2018Q1    1.0
2018Q2    1.0
2018Q3    1.0
2018Q4    1.0
2019Q1    1.0
2019Q2    1.0
2019Q3    1.0
2019Q4    1.0
2020Q1    1.0
2020Q2    1.0
2020Q3    1.0
2020Q4    1.0
2021Q1    1.0
2021Q2    1.0
2021Q3    1.0
2021Q4    1.0
2022Q1    1.0
2022Q2    1.0
2022Q3    1.0
2022Q4    1.0
Freq: Q-DEC, dtype: float64

## Ensemble

In [20]:
pred_now

,DFM,BEM1,BEM2,BEM3,LBVAR,TFM,T-M,VAR,SVR-KNN,CRNN,RF-DVS,S-XGB,LQR,OptimARIMA
TIME,,,,,,,,,,,,,,
2016Q1,0.096766,0.199137,0.093120,0.225933,0.747097,0.476445,0.819795,0.798164,0.350114,-0.282779,0.536181,0.571596,0.303693,0.040466
2016Q2,0.909681,0.892079,0.848110,0.891963,0.351429,1.063370,1.068642,1.132944,1.051205,0.581873,0.772699,1.754219,0.563998,1.266576
2016Q3,0.550106,0.641954,0.563895,0.684774,0.936626,0.953511,0.726452,0.950103,0.454018,0.696878,0.863918,0.928201,0.592652,0.442873
2016Q4,1.131033,0.695008,0.642045,0.712063,0.575754,1.067428,0.872915,1.045621,1.284476,0.960551,0.734647,1.099487,0.769816,1.336816
2017Q1,1.077116,1.119916,1.097867,1.054444,0.647591,1.067023,1.047909,1.113357,1.104790,1.213495,0.818433,0.892966,1.074216,1.447208
2017Q2,0.508361,1.108525,1.126052,1.015314,0.689279,0.932466,0.944884,0.786986,1.029633,0.792337,0.794983,1.191612,0.945407,1.284572
2017Q3,0.737805,0.287701,0.275753,0.399241,0.969641,0.743459,0.834732,0.960132,0.921330,0.675902,0.800451,0.879962,0.712376,0.973802
2017Q4,0.710097,0.952379,0.673771,0.895778,1.387553,0.872120,0.894160,0.694095,0.512125,0.719352,0.743252,0.798802,0.354928,0.478414
2018Q1,1.107998,0.678722,0.808093,0.709687,0.753485,0.977526,1.005438,0.937068,1.214090,0.804410,1.111777,0.712198,0.744322,1.716610


In [21]:
ensemble_now = pred_now * weights

In [22]:
ensemble_now.sum(axis=1)

TIME
2016Q1    0.355409
2016Q2    0.939199
2016Q3    0.713283
2016Q4    0.923404
2017Q1    1.017118
2017Q2    1.019633
2017Q3    0.692918
2017Q4    0.927793
2018Q1    0.808371
2018Q2    0.620427
2018Q3    0.648394
2018Q4    0.419832
2019Q1    0.007044
2019Q2    0.885394
2019Q3    0.539060
2019Q4    0.870227
2020Q1    0.268159
2020Q2   -0.740150
2020Q3    1.249206
2020Q4    1.435228
2021Q1    1.789961
2021Q2    1.075257
2021Q3    1.029155
2021Q4    1.328488
2022Q1    0.734316
2022Q2    0.472800
2022Q3    0.568689
2022Q4   -0.110574
Freq: Q-DEC, dtype: float64

In [23]:
ensemble_now.sum(axis=1).to_excel('after_now.xlsx')

## Test

In [24]:
mean_absolute_error(true, ensemble_now.sum(axis=1))

0.4226569710083269

In [25]:
np.sqrt(mean_squared_error(true, ensemble_now.sum(axis=1)))

0.6725760832611629

In [26]:
max(abs(true - ensemble_now.sum(axis=1)))

2.2598498320923963

# 1Q Ahead

## Load Data

In [3]:
pred_1qa = pd.read_excel('pred_1qa.xlsx', index_col='TIME')

In [4]:
pred_1qa.index = pd.PeriodIndex(pred_1qa.index, freq='M')

In [5]:
pred_1qa = pred_1qa.asfreq('Q-DEC')

In [6]:
errors_1qa = pd.read_excel('error_1qa.xlsx', index_col='TIME')

In [7]:
errors_1qa.index = pd.PeriodIndex(errors_1qa.index, freq='M')

In [8]:
errors_1qa = errors_1qa.asfreq('Q-DEC')

In [9]:
true = pd.read_excel('GDP_true.xlsx', index_col='TIME')

In [10]:
true.index = pd.PeriodIndex(true.index, freq='M')

In [11]:
true = true.asfreq('Q-DEC')

In [12]:
true = true.reindex(errors_1qa.index)

In [13]:
true = true['GDP(SA)']

## Make Weights

In [14]:
weights = pd.DataFrame(index=errors_1qa.index, columns=errors_1qa.columns)

In [15]:
weights.loc[:'2017Q1'] = 1 / len(weights.columns)

In [16]:
weights

,DFM,BEM1,BEM2,BEM3,TFM,T-M,VAR,SVR-KNN,RF-DVS,S-XGB,LQR,OptimARIMA
TIME,,,,,,,,,,,,
2016Q2,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333
2016Q3,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333
2016Q4,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333
2017Q1,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333
2017Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
for i in range(4, len(errors_1qa)):
  weights.iloc[i] = (
      (weights.iloc[i-1] / errors_1qa.iloc[:i-1].std()) * np.exp(-errors_1qa.iloc[i-2]**2 / (2 * errors_1qa.iloc[:i-1].var()))
      / np.sum((weights.iloc[i-1] / errors_1qa.iloc[:i-1].std()) * np.exp(-errors_1qa.iloc[i-2]**2 / (2 * errors_1qa.iloc[:i-1].var())))
  )


In [29]:
weights

,DFM,BEM1,BEM2,BEM3,TFM,T-M,VAR,SVR-KNN,RF-DVS,S-XGB,LQR,OptimARIMA
TIME,,,,,,,,,,,,
2016Q2,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333
2016Q3,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333
2016Q4,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333
2017Q1,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333
2017Q2,0.080403,0.092356,0.073294,0.070216,0.080831,0.119099,0.036021,0.152961,0.116795,0.013504,0.102059,0.06246
2017Q3,0.096827,0.150781,0.100955,0.067585,0.050795,0.208318,0.006548,0.009483,0.057787,0.012235,0.153558,0.085129
2017Q4,0.160934,0.091003,0.061682,0.108387,0.060171,0.148739,0.00474,0.00047,0.017409,0.016235,0.31501,0.015221
2018Q1,0.283954,0.01511,0.009028,0.203926,0.012593,0.024347,0.001041,0.000153,0.002247,0.025464,0.300474,0.121663
2018Q2,0.146702,0.035169,0.032208,0.479443,0.005944,0.008617,0.022306,0.001359,0.006279,0.005419,0.229784,0.026771


In [30]:
weights.sum(axis=1)

TIME
2016Q2    1.0
2016Q3    1.0
2016Q4    1.0
2017Q1    1.0
2017Q2    1.0
2017Q3    1.0
2017Q4    1.0
2018Q1    1.0
2018Q2    1.0
2018Q3    1.0
2018Q4    1.0
2019Q1    1.0
2019Q2    1.0
2019Q3    1.0
2019Q4    1.0
2020Q1    1.0
2020Q2    1.0
2020Q3    1.0
2020Q4    1.0
2021Q1    1.0
2021Q2    1.0
2021Q3    1.0
2021Q4    1.0
2022Q1    1.0
2022Q2    1.0
2022Q3    1.0
2022Q4    1.0
2023Q1    1.0
Freq: Q-DEC, dtype: float64

## Ensemble

In [31]:
pred_1qa

,DFM,BEM1,BEM2,BEM3,TFM,T-M,VAR,SVR-KNN,RF-DVS,S-XGB,LQR,OptimARIMA
TIME,,,,,,,,,,,,
2016Q2,0.521872,0.681736,0.609076,0.409307,1.078356,0.987036,-0.034840,0.855918,0.790735,1.104694,0.649907,0.986338
2016Q3,0.787324,0.452506,0.440791,0.647123,1.046014,0.887491,0.414438,0.503638,0.405793,1.135050,0.688193,1.019892
2016Q4,0.616758,0.463071,0.399063,0.681475,0.554616,0.669545,0.240202,0.690385,0.529389,1.246864,0.630800,0.854478
2017Q1,1.089654,0.916089,0.877486,0.785803,1.343074,1.098352,0.106919,0.183837,0.677625,1.242222,0.985421,1.159281
2017Q2,0.707618,0.469731,0.450203,0.857921,1.048354,1.080670,0.406648,-0.358109,0.409885,1.001211,0.778553,1.237650
2017Q3,0.922133,0.500708,0.449075,0.913355,0.586062,0.716398,0.124685,0.246725,0.452341,0.910792,0.845390,1.218567
2017Q4,0.972545,0.547752,0.462819,0.439255,0.986170,1.057514,0.084091,0.422938,0.509669,1.247796,0.849378,1.120793
2018Q1,0.924909,0.669417,0.613940,0.971331,0.837665,0.797082,0.206457,0.631523,0.666314,0.873010,0.548781,0.772475
2018Q2,0.610973,0.689400,0.651112,0.515040,0.334024,0.493753,0.271466,0.480244,0.574253,0.782322,0.663451,1.002659


In [32]:
ensemble_1qa = pred_1qa * weights

In [33]:
ensemble_1qa.sum(axis=1)

TIME
2016Q2    0.720011
2016Q3    0.702354
2016Q4    0.631387
2017Q1    0.872147
2017Q2    0.584987
2017Q3    0.724848
2017Q4    0.813502
2018Q1    0.789239
2018Q2    0.581865
2018Q3    0.828507
2018Q4    0.567461
2019Q1    0.314858
2019Q2    0.416947
2019Q3    0.578977
2019Q4    0.678050
2020Q1    0.571781
2020Q2   -0.001585
2020Q3    0.109542
2020Q4    1.017285
2021Q1    0.817756
2021Q2    0.994265
2021Q3    0.799872
2021Q4    0.630774
2022Q1    0.811584
2022Q2    0.789680
2022Q3    0.434351
2022Q4    0.502383
2023Q1    0.466874
Freq: Q-DEC, dtype: float64

In [34]:
ensemble_1qa.sum(axis=1).to_excel('after_1qa.xlsx')

## Test

In [35]:
mean_absolute_error(true, ensemble_1qa.sum(axis=1))

0.6017327923795657

In [36]:
np.sqrt(mean_squared_error(true, ensemble_1qa.sum(axis=1)))

0.9164914205471222

In [37]:
max(abs(true - ensemble_1qa.sum(axis=1)))

2.998415328379862